In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Employee')

+-----+-------+------------+-----------+
|  id |  name | department | managerId |
+-----+-------+------------+-----------+
| 101 |  John |     A      |    None   |
| 102 |  Dan  |     A      |    101    |
| 103 | James |     A      |    101    |
| 104 |  Amy  |     A      |    101    |
| 105 |  Anne |     A      |    101    |
| 106 |  Ron  |     B      |    101    |
+-----+-------+------------+-----------+


### Write an SQL query to report the managers with at least five direct reports.
```
+------+
| name |
+------+
| John |
+------+

```

In [3]:
%%sql 

SELECT m.name
FROM employee e 
JOIN employee m 
ON m.id = e.managerId
GROUP BY m.name 
HAVING COUNT(*) >= 5

name
John


# Using Pandas

In [4]:
import pandas as pd 

In [5]:
employee_query = %sql SELECT * FROM employee # type: ignore 
employee_df = employee_query.DataFrame()
employee_df

,id,name,department,managerId
0,101,John,A,NaN
1,102,Dan,A,101.0
2,103,James,A,101.0
3,104,Amy,A,101.0
4,105,Anne,A,101.0
5,106,Ron,B,101.0


In [6]:
employee_df.merge(employee_df, left_on='id', right_on='managerId')

,id_x,name_x,department_x,managerId_x,id_y,name_y,department_y,managerId_y
0,101,John,A,NaN,102,Dan,A,101.0
1,101,John,A,NaN,103,James,A,101.0
2,101,John,A,NaN,104,Amy,A,101.0
3,101,John,A,NaN,105,Anne,A,101.0
4,101,John,A,NaN,106,Ron,B,101.0


In [7]:
## add a suffix for the second table 
joined_df = employee_df.merge(employee_df, left_on='id', right_on='managerId', suffixes=('', '_manager'))
joined_df

,id,name,department,managerId,id_manager,name_manager,department_manager,managerId_manager
0,101,John,A,NaN,102,Dan,A,101.0
1,101,John,A,NaN,103,James,A,101.0
2,101,John,A,NaN,104,Amy,A,101.0
3,101,John,A,NaN,105,Anne,A,101.0
4,101,John,A,NaN,106,Ron,B,101.0


In [8]:
joined_df.groupby('name').filter(lambda x: len(x) >= 5)

,id,name,department,managerId,id_manager,name_manager,department_manager,managerId_manager
0,101,John,A,NaN,102,Dan,A,101.0
1,101,John,A,NaN,103,James,A,101.0
2,101,John,A,NaN,104,Amy,A,101.0
3,101,John,A,NaN,105,Anne,A,101.0
4,101,John,A,NaN,106,Ron,B,101.0


In [9]:
joined_df.groupby('name').filter(lambda x: len(x) >= 5)[['name']]

,name
0,John
1,John
2,John
3,John
4,John


In [10]:
joined_df.groupby('name').filter(lambda x: len(x) >= 5)[['name']].drop_duplicates()

,name
0,John


## Append more data for testing

In [11]:
import numpy as np 

In [12]:
# new rows to be appended
new_data = {
    'id': [107, 108, 109, 110],
    'name': ['Sarah', 'Mike', 'Linda', 'Emily'],
    'department': ['C', 'C', 'C', 'C'],
    'managerId': [np.nan, 107, 107, 107]
}
# save the new rows to a df 
new_rows_df = pd.DataFrame(new_data)
new_rows_df

,id,name,department,managerId
0,107,Sarah,C,NaN
1,108,Mike,C,107.0
2,109,Linda,C,107.0
3,110,Emily,C,107.0


In [13]:
# append new rows
employee_df = pd.concat([employee_df, new_rows_df], ignore_index=True)
employee_df

,id,name,department,managerId
0,101,John,A,NaN
1,102,Dan,A,101.0
2,103,James,A,101.0
3,104,Amy,A,101.0
4,105,Anne,A,101.0
5,106,Ron,B,101.0
6,107,Sarah,C,NaN
7,108,Mike,C,107.0
8,109,Linda,C,107.0
9,110,Emily,C,107.0


In [14]:
joined_df = employee_df.merge(employee_df, left_on='id', right_on='managerId', suffixes=('', '_manager'))
joined_df

,id,name,department,managerId,id_manager,name_manager,department_manager,managerId_manager
0,101,John,A,NaN,102,Dan,A,101.0
1,101,John,A,NaN,103,James,A,101.0
2,101,John,A,NaN,104,Amy,A,101.0
3,101,John,A,NaN,105,Anne,A,101.0
4,101,John,A,NaN,106,Ron,B,101.0
5,107,Sarah,C,NaN,108,Mike,C,107.0
6,107,Sarah,C,NaN,109,Linda,C,107.0
7,107,Sarah,C,NaN,110,Emily,C,107.0


In [18]:
joined_df.groupby('name').filter(lambda x: len(x) >= 5)

,id,name,department,managerId,id_manager,name_manager,department_manager,managerId_manager
0,101,John,A,NaN,102,Dan,A,101.0
1,101,John,A,NaN,103,James,A,101.0
2,101,John,A,NaN,104,Amy,A,101.0
3,101,John,A,NaN,105,Anne,A,101.0
4,101,John,A,NaN,106,Ron,B,101.0


In [19]:
joined_df.groupby('name').filter(lambda x: len(x) >= 5)[['name']].drop_duplicates()

,name
0,John
